# IQVIA NLP - FDA Drug Label Biomarkers

## API Description
Understanding biomarkers for drug efficacy or disease progression is critical in clinical development. Relevant data is often in unstructured text, such as clinical trial records, and drug labels.   For Data Scientists, the FDA Drug Label Biomarkers API solves this problem by offering an API that extracts gene/protein biomarkers from specific regions of FDA Drug labels (Indications and Usages section). Biomarker-disease relationships are provided (e.g.positive, negative); diseases are mapped to MedDRA, and mechanism of action data are also extracted.

## Accessing the API
In order to consume this API, you will first need to Request access to the FDA Drug Label Biomarkers API 
via this link:
https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytJqMAAU/api-marketplaceiqvianlpfdadruglabelbiomarkerspreview .

Please refer to "API Documentation" to learn more about accessing and using the API.

## Notebook Description
This notebook is designed to biomarkers and their related diseases from from openFDA drug product labeling records. Users could specify records of interest by supplying FDA Drug Label document identifiers in the 'FDAID' parameter when posting the request.

### Authorization
The instructions for getting your credentials and the API endpoint URL can be found under the section "Get Started" and "How to use the API" following this link: https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytJqMAAU/api-marketplaceiqvianlpfdadruglabelbiomarkerspreview .

In [1]:
import getpass

# Get URL and credentials from customers
api_endpoint_url = input('Please enter the API URL: ').rstrip('/')

mkp_user = input("Marketplace clientId: ")
mkp_password = getpass.getpass("Marketplace clientSecret: ")
mkp_headers = {'clientId': mkp_user, 'clientSecret': mkp_password}

print("Thanks for inputting URL, your user name and password!")

Please enter the API URL: https://vt.eu-apim-devtest.solutions.iqvia.com/eu/fdadruglabels/api/v2/fdadruglabels/biomarkers
Marketplace clientId: f7fd16f8f6684068a6c17e8eee17fda6
Marketplace clientSecret: ········
Thanks for inputting URL, your user name and password!


### Example: Make a request for a list of known FDA Drug Label documents
FDA Drug Label Biomarker NLP API expects openFDA application number (FDAID) as input parameter. This example shows how to make a request to the API with a list of FDAIDs.


In [2]:
import requests
import time

# Define input FDAIDs
input_fdaids = ['NDA204114', 'ANDA076183']

# Make a request
print("Posting request to extract biomarkers and related diseases from specified FDA Drug Labels documents...")

response = requests.post(api_endpoint_url, headers=mkp_headers, json={'params': {'FDAID': input_fdaids}, 'row_limit': 10})

# Poll the API until results are available
while response.status_code == 202:
    print('Results are not available yet. Waiting 10 seconds before polling again...')
    time.sleep(10)
    # Use the run id from the Post request to get results
    run_identifier = response.json()['id']    
    response = requests.get(url=f'{api_endpoint_url}/{run_identifier}', headers=mkp_headers)
    
# Check the response
if response.status_code == 200:
    print("Success!")
    results_json = response.json()
else:
    raise Exception(f"Unexpected status code: {response.status_code}")

print(f'Results: \n{results_json}\n')

Posting request to extract biomarkers and related diseases from specified FDA Drug Labels documents...
Results are not available yet. Waiting 10 seconds before polling again...
Results are not available yet. Waiting 10 seconds before polling again...
Results are not available yet. Waiting 10 seconds before polling again...
Success!
Results: 
[{'doc_id': 'cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72', 'results': [{'Biomarker': {'logical_column_id': 0, 'value': 'CYP3A4', 'indexed_spans_outer': [[69637, 69644]], 'indexed_spans_inner': [[69637, 69644]], 'text_spans_outer': [[71, 78]], 'text_spans_inner': [[71, 78]]}, 'Relation': {'logical_column_id': 1, 'value': 'inducer', 'indexed_spans_outer': [[69645, 69653]], 'indexed_spans_inner': [[69645, 69653]], 'text_spans_outer': [[79, 87]], 'text_spans_inner': [[79, 87]]}, '[SID] Diseases and Disorder': {'logical_column_id': 2, 'value': 'meddra', 'indexed_spans_outer': [[28552, 28558]], 'indexed_spans_inner': [[28552, 28558]], 'text_spans_outer': [[18, 

Now that we have got the JSON responses from the FDA Drug Label Biomarker API, we could convert the useful information associated with the keys into a pandas dataframe.

In [3]:
import pandas as pd

# initiate an empty dataframe
df = pd.DataFrame()
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.width", 1000)

# Retrieve main results from the JSON response, please note this cell would fail if the request failed in the last step
for inner_results in results_json:
    for result_dict in inner_results['results']:
        df_dict = {}
        for key, value_dict in result_dict.items():                                                                                                                                                                                                                                                                                                                           
            df_dict[key] = value_dict['value']                                                                                                                                                                                                                                                                                                                                
        df_dict['Doc Id'] = inner_results['doc_id']                                                                                                                                                                                                                                                                                                                           
        df = pd.concat([df, pd.DataFrame.from_records([{**df_dict}])], ignore_index=True)                                                                                                                                                                                                                                                                                     

# Print the DataFrame                                                                                                                                                                                                                                                                                                                                                         
df.head(10)

,Biomarker,Relation,[SID] Diseases and Disorder,[NID] Diseases and Disorder,[PT] Diseases and Disorder,Section,FDAID,Label Revision Date,text,Doc Id
0,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,"highly emetogenic cancer chemotherapy, includi...",cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
1,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,... repeat courses of moderately emetogenic ca...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
2,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,"highly emetogenic cancer chemotherapy, includi...",cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
3,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,... repeat courses of moderately emetogenic ca...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
4,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Drug Interaction & Indications & Usage,ANDA076183,20211006,"highly emetogenic cancer chemotherapy, includi...",cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
5,CYP3A4,inducer,meddra,10028997,Neoplasm malignant,Drug Interaction & Indications & Usage,ANDA076183,20211006,... repeat courses of moderately emetogenic ca...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
6,CYP3A4,inducer,meddra,10047700,Vomiting,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,... indicated for the prevention of nausea and...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
7,CYP3A4,inducer,meddra,10047700,Vomiting,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,... indicated for the prevention of nausea and...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
8,CYP3A4,inducer,meddra,10047700,Vomiting,Drug Interaction & Indications & Usage,ANDA076183,20211006,... indicated for the prevention of nausea and...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72
9,CYP3A4,inducer,meddra,10066962,Procedural nausea,Clinical Pharmacology & Indications & Usage,ANDA076183,20211006,... indicated for the prevention of postoperat...,cdb2c4b4-a20a-3f8f-e053-2a95a90aaf72


### Optional: Make a request to openFDA drug product labeling API to get FDAIDs associated with a specific criteria

Search URL for the openFDA drug product labeling API that allows user to search for the FDAId's of 10 documents. Search can be customized by modifying the search parameters. In this example, the search will return 10 FDAIds of FDA Drug Labels about prescription drugs with indications and usage for cancer.


In [4]:
import json

# Search URL for the openFDA drug product labeling API
# Search can be customized by modifying the search parameters in the URL

url = "https://api.fda.gov/drug/label.json?search=openfda.product_type:prescription+AND+indications_and_usage:'cancer'&limit=10"

# load in JSON reponse from the openFDA API
response = requests.get(url)
body = json.loads(response.text)

# initialize list for FDAID
fdaids = []

# find the openfda application number in response (if exists) and add to FDAID list
for i in body['results']:
    try:
        #extract the document id (ANDA, ANADA, BA, BLA, BN, BP, DMF,K, MIF, NDA, NADA, P or VM)
        x = i["openfda"]
        fdaids.append(x['application_number'][0])
    except KeyError:
        print(f"No openFDA application number found for response results id: {i['id']}")
        continue

print(fdaids)

No openFDA application number found for response results id: ae413477-0e46-4763-8005-091f81dd6070
['NDA204114', 'ANDA076183', 'ANDA075798', 'ANDA040584', 'NDA050629', 'ANDA204345', 'ANDA212399', 'NDA021116', 'BLA761310']


That's it! Hope you find this tutorial useful! Bye!